# FIR-Filterentwurf am Beispiel

In [ ]:
!conda install -y -q numpy
!conda install -y -q matplotlib
!conda install -y -q scipy

import numpy as np
import matplotlib.pyplot as plt
import scipy.signal

import coeff2header

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

## Schritt 1: Spezifikation der Filtereigenschaften

In [ ]:
# Filter-Spezifikation
fsample = 8000          # Abtastfrequenz
fp      = 1200          # Passband-Eckfrequenz
fs      = 1800          # Stopband-Eckfrequenz
nyq     = fsample/2     # Nyquist-Frequenz

fcutoff = fp+(fs-fp)/2   # Cutoff-Frequenz wird genau zwischen Passband- und Stopband-Eckfrequenz gelegt (für Tiefpass)
#fcutoff = fp-(fp-fs)/2   # Cutoff-Frequenz wird genau zwischen Passband- und Stopband-Eckfrequenz gelegt (für Hochpass)

## Schritt 2a: Entwurf mit Hann-Fenster

In [ ]:
# Anzahl an Filterkoeffizienten
numtaps = 51

# Berechnung der Filterkoeffizienten für ein Tiefpass mit Hann-Fenster
# Dokumentation: https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.firwin.html
taps = scipy.signal.firwin(numtaps, (fcutoff/nyq), pass_zero="lowpass", window="hann")

## Schritt 2b: Entwurf mit Kaiser-Fenster

In [ ]:
# Dämpfung im Stoppband (in dB)
attn = 60

# Berechnung der Anzahl an Filterkoeffizienten sowie Beta-Wert für
# eine gewünschte Dämpfung und Übergangbereichsbreite
# Dokumentation: https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.kaiserord.html
numtaps, beta = scipy.signal.kaiserord(attn, (fs-fp)/nyq)

# Berechnung der Filterkoeffizienten
taps = scipy.signal.firwin(numtaps, (fcutoff/nyq), window=('kaiser', beta))

## Schritt 3: Verifikation der Filtereigenschaften

In [ ]:
# Berechnung der Frequenzantwort aus den Filterkoeffizienten
w, Hd = scipy.signal.freqz(taps)

# Grafische Darstellung
plt.figure(figsize=(14,6))
plt.plot((w/np.pi)*(fsample/2), 20*np.log10(np.abs(Hd)))
plt.xlabel('Frequenz (Hz)')
plt.ylabel('Verstärkung (dB)')
plt.title('Frequenzantwort (Filterkoeffizienten: %d)' % (numtaps));
plt.grid()

## Schritt 4: Export der Koeffizienten

In [ ]:
# Die Filterkoeffizienten werden als C-Header-Datei in 'fir_export.h' abgelegt und können direkt in ein C-Programm übernommen werden
coeff2header.FIR_header('fir_export.h', taps)